### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
    included_vehicle_type=[
        "van",
        "bus",
        "truck",
        "NaN",
        "pickup",
        "other",
        "sedan",
        "taxi",
        "motorbike",
        "suv",
        "hatchback",
    ],
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

                 camera       timestamp_precise    xmid   ymid    lifetime plate_number vehicle_type  timestamp_unix  timestamp_unix_end
2340  mbk-14-11-vehicle 2024-04-28 02:08:07.953  1383.0  118.0    1.480000       302165    motorbike      1714270087          1714270088
2341  mbk-14-11-vehicle 2024-04-28 02:08:56.179  1394.5  110.0    1.000000      4กน6102    hatchback      1714270136          1714270137
2336  mbk-14-11-vehicle 2024-04-28 02:12:38.481  1299.5  118.0    1.001000      1มช1142         taxi      1714270358          1714270359
2337  mbk-14-11-vehicle 2024-04-28 02:14:02.737  1330.5  106.5    1.360000       ฮอ1753          van      1714270442          1714270443
2331  mbk-14-11-vehicle 2024-04-28 02:14:58.722  1399.5  139.0    0.000000      1กผ1251        sedan      1714270498          1714270498
2332  mbk-14-11-vehicle 2024-04-28 02:15:01.245  1399.5  139.0    1.644956      1กผ1251        sedan      1714270501          1714270502
2333  mbk-14-11-vehicle 2024-04-28 02:17:

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [4]:
df_vehicle["lifetime"]

2340    1.480
2341    1.000
2336    1.001
2337    1.360
2331    0.000
        ...  
53      1.000
54      1.518
39      1.479
40      0.760
19      1.240
Name: lifetime, Length: 252, dtype: float64

In [5]:
df_vehicle = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle = match_cross_to_vehicle(
    df_cross,
    df_vehicle,
)


df_vehicle = df_vehicle[
    (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
]
df_vehicle = df_vehicle[
    [
        *conf.VEHICLE_INTEREST_SNAPSHOT,
        "cross_count",
        "reverse_count",
        "original_video_name",
        # "frame_no",
    ]
]

df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
df_vehicle

,camera,timestamp_precise,xmid,ymid,lifetime,plate_number,vehicle_type,cross_count,reverse_count,original_video_name
1867,mbk-14-11-vehicle,2024-04-28 06:02:29.843,1388.5,120.5,18.875,41360,sedan,2,0,mbk-14-11-vehicle_1714284001.mp4
1860,mbk-14-11-vehicle,2024-04-28 06:05:35.235,1406.5,126.0,28.751,6กม9678,sedan,1,1,mbk-14-11-vehicle_1714284301.mp4
1861,mbk-14-11-vehicle,2024-04-28 06:06:08.106,1403.0,140.5,15.516,มฎ5222,suv,4,0,mbk-14-11-vehicle_1714284301.mp4
1862,mbk-14-11-vehicle,2024-04-28 06:06:28.620,1535.0,187.0,32.111,3กฐ7375,hatchback,2,0,mbk-14-11-vehicle_1714284301.mp4
1863,mbk-14-11-vehicle,2024-04-28 06:08:58.459,1892.5,214.0,51.865,สก573,tuktuk,0,0,mbk-14-11-vehicle_1714284301.mp4
1850,mbk-14-11-vehicle,2024-04-28 06:09:36.567,1416.5,179.0,30.955,ทส4264,sedan,1,0,mbk-14-11-vehicle_1714284301.mp4
1851,mbk-14-11-vehicle,2024-04-28 06:10:12.521,1630.0,196.5,35.316,4ขภ5400,pickup,1,0,mbk-14-11-vehicle_1714284601.mp4
1852,mbk-14-11-vehicle,2024-04-28 06:10:36.875,1426.5,175.5,7.878,สก2722,tuktuk,0,0,mbk-14-11-vehicle_1714284601.mp4
1853,mbk-14-11-vehicle,2024-04-28 06:12:27.004,1425.0,174.0,22.594,2ขจ8645,pickup,0,3,mbk-14-11-vehicle_1714284601.mp4
1854,mbk-14-11-vehicle,2024-04-28 06:13:05.633,1421.5,168.5,31.511,5กน9372,pickup,1,0,mbk-14-11-vehicle_1714284601.mp4


## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?